<a href="https://colab.research.google.com/github/ameerfayiz/ML-notebooks/blob/main/finetune_GPT2_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://medium.com/ai-innovation/beginners-guide-to-retrain-gpt-2-117m-to-generate-custom-text-content-8bb5363d8b7f

In [2]:
!git clone https://github.com/nshepperd/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 429, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 429 (delta 20), reused 77 (delta 11), pack-reused 291
Receiving objects: 100% (429/429), 4.47 MiB | 5.69 MiB/s, done.
Resolving deltas: 100% (216/216), done.


In [4]:
!pip install fire>=0.1.3
!pip install regex==2017.4.5
!pip install requests==2.21.0
!pip install tqdm==4.31.1
!pip install toposort==1.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!cd gpt-2 && pip install -r  requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
base = "355M"

In [78]:
!python gpt-2/download_model.py $base

Fetching checkpoint: 1.00kit [00:00, 860kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:01, 555kit/s]                                                    
Fetching hparams.json: 1.00kit [00:00, 976kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [04:44, 4.99Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 9.33Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:01, 490kit/s]                                                  
Fetching vocab.bpe: 457kit [00:01, 368kit/s]                                                        


In [8]:
import csv
import re


filepath = 'sentence_to_chunks.csv'
mlist = []
end_of_text_token = "<|endoftext|>"

with open(filepath) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    x = 0
    for row in csv_reader:
        _str = """Below is a given INPUT as text and OUTPUT is its  multiple non dividable  sentences such that each sentence contain only single knowledge meaning in it. 
        
        INPUT :\n{text}
        OUTPUT :\n{out}""".replace("{text}",re.sub('\s+',' ',row[0])).replace("{out}",re.sub('\s+',' ',row[1]))
        mlist.append(re.sub('\s+',' ',_str))

In [9]:
for item in mlist:
  with open("data.txt", 'a') as f:
    f.write("\n\n\n" + item.replace("INPUT :","\n\nINPUT :").replace("OUTPUT : ","\nOUTPUT :\n").replace('", "','",\n"') + "\n<|endoftext|>")
     

In [79]:
!python gpt-2/src/encode.py data.txt data.npz --model_name $base


Reading files
100% 1/1 [00:00<00:00, 4760.84it/s]
Writing data.npz


In [80]:
!rm -r checkpoint/run1

In [ ]:
!python gpt-2/src/train.py --dataset data.npz --batch_size 2 --model_name $base --learning_rate 0.00002 --sample_every 50 --sample_num 3 --sample_length 1023 --save_every 100

2023-02-10 12:40:16.116954: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.random.categorical` instead.
Instructions for updating:
Use `tf.random.categorical` instead.
Using Adam optimizer
Loading checkpoint models/124M/model.ckpt
Loading dataset...
100% 1/1 [00:00<00:00, 542.18it/s]
dataset has 11489 tokens
Training...
[1 | 2.02] loss=1.82 avg=1.82
[2 | 2.85] loss=1.31 avg=1.56
[3 | 3.69] loss=1.26 avg=1.46
[4 | 4.52] loss=1.33 avg=1.43
[5 | 5.35] loss=1.42 avg=1.43
[6 | 6.19] loss=1.37 avg=1.42
[7 | 7.02] loss=1.28 avg=1.40
[8 | 7.86] loss=1.32 avg=1.39
[9 | 8.70] loss=1.28 avg=1.37
[10 | 9.54] loss=1.24 avg=1.36
[11 | 10.39] loss=1.27 avg=1.35
[12 | 11.23] loss=1.15 avg=1.33
[13 | 12.07] loss=1.24 avg=1.33
[14 |

In [59]:
ckpt = "model-400"
newmodel = "models/STC"
f1 = f"{ckpt}.data-00000-of-00001"
f2 = f"{ckpt}.index"
f3 = f"{ckpt}.meta"
f4 = f"checkpoint"

f11 = f"model.ckpt.data-00000-of-00001"
f21 = f"model.ckpt.index"
f31 = f"model.ckpt.meta"

!cp checkpoint/run1/$f1 $newmodel/$f11
!cp checkpoint/run1/$f2 $newmodel/$f21
!cp checkpoint/run1/$f3 $newmodel/$f31
!cp models/$base/$f4 $newmodel/$f4

!cp models/$base/encoder.json $newmodel/encoder.json
!cp models/$base/hparams.json $newmodel/hparams.json
!cp models/$base/vocab.bpe $newmodel/vocab.bpe

In [66]:
##INFERENCE

!python gpt-2/src/generate_unconditional_samples.py --temperature 1 --top_k 0 --top_p 1.0 --model_name STC

2023-02-10 11:51:25.229092: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.random.categorical` instead.
Instructions for updating:
Use `tf.random.categorical` instead.
======================================== SAMPLE 1 ========================================
Wonder if my Internet connection is working.I tried using a standard DVD burner but it is not working.I tried playing some songs on the iPod touch but it is not working.I tried playing a few songs on the radio but it is not working.I tried listening to music on the iPods but it is not working.I tried watching TV shows on the iPods but it is not working.I tried taking pictures on the iPods but it is not working.I tried downloading movies on the iPods but it is n

In [77]:
!python gpt-2/src/interactive_conditional_samples.py --temperature 0.1 --top_k 40 --top_p 1.0 --model_name 'STC'

2023-02-10 12:11:59.127331: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.random.categorical` instead.
Instructions for updating:
Use `tf.random.categorical` instead.
Model prompt >>> Below is a given INPUT as text and OUTPUT is its multiple non dividable sentences such that each sentence contain only single knowledge meaning in it.\n\nINPUT : anand came home today and we had a great time . went to kochi for dinner. \nOUTPUT :
======================================== SAMPLE 1 ========================================
 A few days ago, I had a job as a marketing manager at a multinational corporation. I was responsible for creating and implementing a large-scale marketing strategy for the company.  I also had the opp